In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
print("📂 Loading optical properties...")

# Update this path to your optical properties folder
optical_props_path = r"C:\Users\nmuthan\Desktop\OpProps-Nanda"

# Load materials
def load_materials(base_path):
    materials = {}
    
    # Load Void
    void_df = pd.read_csv(Path(base_path) / "Void.csv")
    materials['void'] = void_df
    
    # Load CdTe
    cdte_df = pd.read_csv(Path(base_path) / "CdTe-OpProp.csv")
    materials['cdte'] = cdte_df
    
    # Load Oxide
    oxide_df = pd.read_csv(Path(base_path) / "NTVE_JAW.csv")
    materials['oxide'] = oxide_df
    
    # Load Silicon
    si_df = pd.read_csv(Path(base_path) / "Si_JAW.csv")
    materials['si'] = si_df
    
    return materials

materials = load_materials(optical_props_path)

# Convert to TensorFlow tensors
materials_tf = {}
materials_tf['wavelengths'] = tf.constant(materials['void']['Wavelength (nm)'].values, dtype=tf.float32)
materials_tf['void_N'] = tf.constant(materials['void']['n'].values + 1j * materials['void']['k'].values, dtype=tf.complex64)
materials_tf['cdte_N'] = tf.constant(materials['cdte']['n'].values + 1j * materials['cdte']['k'].values, dtype=tf.complex64)
materials_tf['oxide_N'] = tf.constant(materials['oxide']['n'].values + 1j * materials['oxide']['k'].values, dtype=tf.complex64)
materials_tf['si_N'] = tf.constant(materials['si']['n'].values + 1j * materials['si']['k'].values, dtype=tf.complex64)

print(f"✅ Materials loaded: {len(materials_tf['wavelengths'])} wavelengths from {materials_tf['wavelengths'][0]:.1f} to {materials_tf['wavelengths'][-1]:.1f} nm")



📂 Loading optical properties...
✅ Materials loaded: 697 wavelengths from 211.3 to 1687.7 nm


In [4]:
@tf.function
def bruggeman_ema_tf_fixed(M1, M2, c):
    """Fixed Bruggeman EMA implementation"""
    c = tf.cast(c, tf.complex64)
    N1 = tf.cast(M1, tf.complex64)
    N2 = tf.cast(M2, tf.complex64)
    p = N1 / N2
    three, one = 3.0, 1.0
    b = 0.25 * ((three * c - one) * ((one / p) - p) + p)
    z = b + tf.sqrt(b * b + 0.5)
    e = z * N1 * N2
    e1, e2 = tf.math.real(e), tf.math.imag(e)
    mag = tf.sqrt(e1 * e1 + e2 * e2)
    n = tf.sqrt((mag + e1) / 2)
    k = tf.sqrt((mag - e1) / 2)
    N_ema = tf.complex(n, k)
    return N_ema

In [5]:
@tf.function
def build_structure_tf_all_cases(case_int, thick_bulk, thick_surf, thick_nuc, vf_surf, vf_nuc):
    """Build structure for all 4 cases with proper layer ordering"""
    
    # Get base materials
    void_N = materials_tf['void_N']
    cdte_N = materials_tf['cdte_N']
    oxide_N = materials_tf['oxide_N']
    si_N = materials_tf['si_N']
    
    # Create EMA materials
    surf_ema_N = bruggeman_ema_tf_fixed(cdte_N, void_N, vf_surf)
    nuc_ema_N = bruggeman_ema_tf_fixed(cdte_N, void_N, vf_nuc)
    
    # Fixed oxide thickness
    oxide_thick = tf.constant(1.75, dtype=tf.float32)
    
    # Case-specific structure building
    if case_int == 0:  # Case2_NoEMA: Void/CdTe/Oxide/Si
        layers = [void_N, cdte_N, oxide_N, si_N]
        thicknesses = [thick_bulk, oxide_thick]
        
    elif case_int == 1:  # Case1_NucOnly: Void/CdTe/NucEMA/Oxide/Si
        layers = [void_N, cdte_N, nuc_ema_N, oxide_N, si_N]
        thicknesses = [thick_bulk, thick_nuc, oxide_thick]
        
    elif case_int == 2:  # Case3_SurfOnly: Void/SurfEMA/CdTe/Oxide/Si
        layers = [void_N, surf_ema_N, cdte_N, oxide_N, si_N]
        thicknesses = [thick_surf, thick_bulk, oxide_thick]
        
    elif case_int == 3:  # Case4_BothEMAs: Void/SurfEMA/CdTe/NucEMA/Oxide/Si
        layers = [void_N, surf_ema_N, cdte_N, nuc_ema_N, oxide_N, si_N]
        thicknesses = [thick_surf, thick_bulk, thick_nuc, oxide_thick]
    
    # Convert to tensors
    layer_stack = tf.stack(layers, axis=0)
    thickness_array = tf.stack(thicknesses, axis=0)
    
    return layer_stack, thickness_array

In [6]:
@tf.function
def snells_law_tf(N_stack, aoi_rad):
    """Calculate angles in each layer using Snell's law"""
    num_layers = tf.shape(N_stack)[0]
    num_wl = tf.shape(N_stack)[1]
    
    angles = tf.TensorArray(dtype=tf.complex64, size=num_layers)
    first_angle = tf.cast(aoi_rad, tf.complex64)
    angles = angles.write(0, tf.fill([num_wl], first_angle))
    
    def body(i, angles):
        sin_prev = tf.sin(angles.read(i-1))
        n_ratio = N_stack[i-1] / N_stack[i]
        sin_curr = n_ratio * sin_prev
        angle_curr = tf.asin(sin_curr)
        angles = angles.write(i, angle_curr)
        return i+1, angles
    
    i = tf.constant(1)
    cond = lambda i, angles: i < num_layers
    _, angles = tf.while_loop(cond, body, [i, angles])
    
    return angles.stack()

In [7]:
@tf.function
def fresnel_coefficients_tf(N_stack, angles):
    """Calculate Fresnel coefficients at all interfaces"""
    n1 = N_stack[:-1]  # Incident media
    n2 = N_stack[1:]   # Transmitted media
    t1 = angles[:-1]   # Incident angles
    t2 = angles[1:]    # Transmitted angles
    
    cos1 = tf.cos(t1)
    cos2 = tf.cos(t2)
    
    # s-polarization
    ds = n1 * cos1 + n2 * cos2
    rs = (n1 * cos1 - n2 * cos2) / ds
    ts = (2 * n1 * cos1) / ds
    
    # p-polarization
    dp = n2 * cos1 + n1 * cos2
    rp = (n2 * cos1 - n1 * cos2) / dp
    tp = (2 * n1 * cos1) / dp
    
    return rs, rp, ts, tp

In [8]:
@tf.function
def scattering_matrix_tf_fixed(N_stack, angles, thicknesses, wavelengths, r, t):
    """Fixed transfer matrix implementation"""
    
    # Get internal layers (exclude ambient and substrate)
    N_layers = N_stack[1:-1]  # Internal layers only
    angles_layers = angles[1:-1]  # Corresponding angles
    
    num_internal = tf.shape(N_layers)[0]
    
    # Wave vector
    k0 = 2 * np.pi / wavelengths
    k0_complex = tf.cast(k0, tf.complex64)
    thicknesses_complex = tf.cast(thicknesses, tf.complex64)
    
    # Match thicknesses to internal layers
    thicknesses_internal = thicknesses_complex[:num_internal]
    
    # Phase factors
    phase_factors = (k0_complex[None, :] * 
                    N_layers * 
                    thicknesses_internal[:, None] * 
                    tf.cos(angles_layers))
    
    # Initialize with first interface
    inv_t = 1.0 / t[0]
    r_over_t = r[0] / t[0]
    
    S11, S12 = inv_t, r_over_t
    S21, S22 = r_over_t, inv_t
    
    # Propagate through each internal layer
    for layer_idx in tf.range(num_internal):
        # Phase propagation
        exp_neg = tf.exp(-1j * phase_factors[layer_idx])
        exp_pos = tf.exp(1j * phase_factors[layer_idx])
        
        # Interface at end of this layer
        interface_idx = layer_idx + 1
        inv_t_next = 1.0 / t[interface_idx]
        r_over_t_next = r[interface_idx] / t[interface_idx]
        
        # Transfer matrix for this layer + interface
        PI11 = exp_neg * inv_t_next
        PI12 = exp_neg * r_over_t_next
        PI21 = exp_pos * r_over_t_next
        PI22 = exp_pos * inv_t_next
        
        # Matrix multiplication
        S11_new = S11 * PI11 + S12 * PI21
        S12_new = S11 * PI12 + S12 * PI22
        S21_new = S21 * PI11 + S22 * PI21
        S22_new = S21 * PI12 + S22 * PI22
        
        S11, S12, S21, S22 = S11_new, S12_new, S21_new, S22_new
    
    return tf.stack([[S11, S12], [S21, S22]], axis=0)

In [9]:
@tf.function
def se_simulation_tf_complete(layer_stack, thicknesses, aoi_deg=70.0):
    """Complete SE simulation with fixed sign conventions"""
    aoi_rad = aoi_deg * np.pi / 180.0
    
    # Calculate angles
    angles = snells_law_tf(layer_stack, aoi_rad)
    
    # Calculate Fresnel coefficients
    rs, rp, ts, tp = fresnel_coefficients_tf(layer_stack, angles)
    
    # Calculate scattering matrices
    Ss = scattering_matrix_tf_fixed(layer_stack, angles, thicknesses, 
                                   materials_tf['wavelengths'], rs, ts)
    Sp = scattering_matrix_tf_fixed(layer_stack, angles, thicknesses, 
                                   materials_tf['wavelengths'], rp, tp)
    
    # Total reflection coefficients
    Rs = Ss[1, 0] / Ss[0, 0]
    Rp = Sp[1, 0] / Sp[0, 0]
    
    # FIXED: Correct rho calculation and sign conventions
    rho = tf.math.conj(Rp / Rs)  # Use conjugation for correct convention
    
    psi = tf.math.atan(tf.math.abs(rho))
    delta = tf.math.angle(rho)
    
    # Calculate N, C, S parameters
    cos_2psi = tf.cos(2 * psi)
    sin_2psi = tf.sin(2 * psi)
    cos_delta = tf.cos(delta)
    sin_delta = tf.sin(delta)
    
    N = cos_2psi
    C = sin_2psi * cos_delta
    S = sin_2psi * sin_delta
    
    # Ensure real output
    N_real = tf.math.real(N)
    C_real = tf.math.real(C)
    S_real = tf.math.real(S)
    
    return tf.stack([N_real, C_real, S_real], axis=0)

In [10]:
def calculate_sigma_weighted(N_model, N_exp, C_model, C_exp, S_model, S_exp, n, m, var=0.001):
    """Your exact sigma loss function"""
    N_model = np.asarray(N_model)
    N_exp = np.asarray(N_exp)
    C_model = np.asarray(C_model)
    C_exp = np.asarray(C_exp)
    S_model = np.asarray(S_model)
    S_exp = np.asarray(S_exp)
    
    resid_N = (N_model - N_exp)**2 / var
    resid_C = (C_model - C_exp)**2 / var
    resid_S = (S_model - S_exp)**2 / var
    
    total = np.sum(resid_N + resid_C + resid_S)
    sigma = np.sqrt(total / (3*n - m))
    return sigma